# **Database Builder Example - Charleston, USA**

In this notebook we are going to create a complete **database** for our FloodAdapt model in Charleston, USA. 
To do so we use the in FloodAdapt integrated **Database builder**, which allows for a quick and easy database creation!
We presume that you have already created a functioning [**Delft-FIAT**](https://github.com/Deltares/Delft-FIAT) and [**SFINCS model**](https://github.com/Deltares/SFINCS) and we"ll go from there. 


## **Step 0**: Import packages

In [ ]:
# Import packages
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from shapely.geometry import Polygon
import toml

import flood_adapt.adapter.fiat_adapter as fiat
import flood_adapt.database_builder.database_builder as db
from flood_adapt.objects.forcing.tide_gauge import TideGaugeSource
from flood_adapt.objects.forcing.timeseries import Scstype
from flood_adapt import unit_system as us
%matplotlib inline

## **Step 1**: Explore the Delft-FIAT model

The study area is in **Charleston, USA**, a coastal city on the East Coast of the United States. 
So, let"s first inspect the exposure data in the **Delft-FIAT model** to get an understanding of our study area.  

We initiate the **FiatAdapter** in FloodAdapt with the model root of the Delft-FIAT model and explore the exposure data in the map.

In [ ]:
# Initiate the FiatAdapter class from FloodAdapt and plot the Fiat model. 
# Set up the settings for the database
root = Path("../../_data/examples").resolve()
fn_fiat = Path(root / "fiat") 
fa = fiat.FiatAdapter(
    model_root =  fn_fiat
    )

# Get the geodataframe with exposure data
gdf = fa._model.exposure.get_full_gdf(fa._model.exposure.exposure_db)

# Plot the region and the secondary_object_types of the exposure data
m = gdf.explore(column="secondary_object_type", name="Exposure types")
m

## **Step 2**: Database Builder Configuration - Build a **simple** FloodAdapt model

The database builder needs simply a **configuration file** as **input** to create the database. The configuration file is a simple text file and can be created manually. In the next steps we will show you how you can create this configuration file. 
So, now that we got an understanding of our study area and our exposure data we are ready to build our base **configuration file** for the **DatabaseBuilder**!

The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

In this step we will look at the basic model parameters.  
- `name`: The name of the database to be created
- `database_path`: The path where to save the database (if not provided, a database folder will be created at the location of this condig file)
- `sfincs_overland`: A dictinary of the overland sfincs model with the entries: file path, reference system (eg. "MSL") 
- `fiat`: The  path to the FIAT model
- `unit_system`: metric or imperial  
* *[gui]* The values below define the GUI variables. They are used as the maximum values to scale the mapbox layers. Same units as in the models should be used. You can adjust the values, so that they are sensible with respect to your data.
- `max_flood_depth`= The maximum flood depth value (m,ft)
- `max_aggr_dmg` = The maximum aggregation damage
- `max_footprint_dmg`= The maximum footprint damage
- `max_benefits` = The maximum benefits

You can either create the configuration file manually with a dictionary or use the integrated classes in FloodAdapt for a more streamlined approach. 
We show you both options below.

### **Step 2a**: Create a configuration file from the Database Builder classes

**Important!**  
The water level reference should be set to the reference of your DEM. You can create a reference system manually as shown below or fetch this information from a close-by tide gauge. This will be shown in the more advanced options. 

In [ ]:
config_model = db.ConfigModel(
    name="charleston_example3",
    database_path= str((root /"output"/"Database").absolute()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    sfincs_overland=db.FloodModel(
        name=str((root / "sfincs").absolute()),
        reference="MSL",
    ),
    fiat=str((root / "fiat").absolute()),
)

### **Step 2b**: Create a configuration file from a dictionary

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 

# all paths should be provided with forward slashes (/)
config = {
"name": "charleston_example1",
"database_path": "Database",
"sfincs_overland" : { 
    "name":"sfincs",
    "reference":"MSL"
    },
"fiat" : "fiat",
"unit_system" :"imperial",
"gui": {
"max_flood_depth": 5,
"max_aggr_dmg" : 10000000,
"max_footprint_dmg": 250000,
"max_benefits" : 50000000}
}

# Save the configuration file
with open(root /"database_builder"/"db_config.toml", "w") as f:
    toml.dump(config, f)

## **Step 2c**: Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 2a** - in which we generated the configuration using the FloodAdapt classes. After we are going to run **Option 2b** - in which we manually created a configuration file from a dictionary. 

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

# Run Option 2b -  DB-builder config from dictionary
config_path = root / "output" / "db_config.toml"
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

Now you created a complete FloodAdapt Database. You can open the databases in the GUI and explore them further. 

<div>
<img src="../_static/images/example_chaleston1.png" width="900"/>
</div>

## **Step 3**: Database Builder Configuration - Build an **advanced** FloodAdapt model

In the previous step we created a simple FloodAdapt model. In this ex step we look at the more advanced configurations and will create a comprehensive FloodAdapt model. 

The configuration file consists of:
1. **Basic model parameters** to create a simple FloodAdapt database.
2. **Advance model parameters** to create a more complex FloodAdapt database.

Additionally, to the basic model parameters (see step 2), you have the option for the following additions to create a more complex model:  
- `probabilistic_set`: The file path to a probabilistic dataset
- `infographics`: Include infographics in output. Boolean Tue/ False
- `return_periods`: A list of return periods. Integers or Floats

*Sfincs*

- `sfincs_offshore`:  A dictinary of the offshore sfincs model with the entries: file path, reference system (eg. "MSL") 
- `dem`: A dictinary of DEM with the entries: file path, units
- `excluded_datums`: A list of datums <span style="color:red;">double check</span>
- `scs`: A dictinary of the SCS (Soil Conservation Service) rainfall with the entries: file path, scs type
- `tide_gauge`: A dictinary of tide gauges with the entries: source, max_distance(dict: values, units)
- `cyclones`: Boolean (True/False)
- `cyclone_basin`:  <span style="color:red;">double check</span>
- `obs_point`: A list of dictionaries with the entries: name, description, lat, lon

*FIAT*

- `aggregation_areas`: A list of dictionaries with the entries: name, file path, field_name. Aggregates the exposure into larger spatial groups o summarize impacts on larger scale. 
- `building_footprints`:  <span style="color:red;">double check</span>
- `fiat_buildings_name`: The name of the exposure type for buildings. Default is set to **"buildings"**
- `fiat_roads_name`:  The name of the exposure type for roads. Default is set to **"roads"**
- `bfe`: The base flood elevation model. A list of dictionaries with the entries: name, file path, field_name.
- `svi`: The social vulnerability index. A dictionary with the entries: file path, field_name, threshold
- `road_width`: The width of the road. Integer or Float

*NOTE!* These parameers are **additional** to the basic parameters, so don"t forge to add these to your configuration as well!

### **Step 3a**: Create a configuration file from the Database Builder classes

In [ ]:
config_model = db.ConfigModel(
    name="charleston_example_advanced1",
    database_path= str((root /"database_buider"/"Database").absolute()),
    unit_system= db.UnitSystems.imperial,
    gui=db.GuiConfigModel(
        max_flood_depth=5,
        max_aggr_dmg=1e6,
        max_footprint_dmg=250000,
        max_benefits=5e6,
    ),
    infographics=True,
    probabilistic_set=str(Path(root /  "databasebuilder" / "test_set")), #TODO fix final path
    return_periods=[1, 2, 5, 10, 25, 50, 100],
    sfincs_overland=db.FloodModel(
        name=str((root / "sfincs").absolute()),
        reference="MSL",
    ),
    sfincs_offshore=db.FloodModel(
        name=str(Path(root / "databasebuilder" / "offshore")),  #TODO fix final path
        reference="MSL",
        vertical_offset=us.UnitfulLength(
            value=0.33, units=us.UnitTypesLength.feet
        )),
    dem=db.DemModel(
        filename=str(Path(root / "databasebuilder" / "charleston_14m.tif")),  #TODO fix final path
        units=us.UnitTypesLength.meters,
    ),
    excluded_datums=["NAVD88"],
    #references=db.WaterlevelReferenceModel(
    #    reference="MSL",
    #    datums=[
    #        db.DatumModel(name="MSL", height=us.UnitfulLength(value=0, units=us.UnitTypesLength.meters)),
    #        db.DatumModel(name="NAVD88", height=us.UnitfulLength(value=1, units=us.UnitTypesLength.meters))
    #    ]
    #),
    slr_scenarios=db.SlrScenariosModel(
        file=str(Path(root / "databasebuilder" / "slr.csv")),
        relative_to_year=2020,
    ),
    scs=db.SCSModel(
        file=str(Path(root / "databasebuilder" / "scs_rainfall.csv")),
        type=Scstype.type3,
    ),
    tide_gauge=db.TideGaugeConfigModel(
        source=db.TideGaugeSource.noaa_coops,
        max_distance=us.UnitfulLength(
            value=100, units=us.UnitTypesLength.miles
        ),
    ),
    cyclones=True,
    cyclone_basin=db.Basins.NA,
    obs_point=[
        db.ObsPointModel(
            name="ashley_river",
            description="Ashley River - James Island Expy",
            lat=32.7765,
            lon=-79.9543,
        ),
        db.ObsPointModel(
            name=8665530,
            description="Charleston Cooper River Entrance",
            ID=8665530,
            lat=32.78,
            lon=-79.9233,
        ),
    ],
    fiat=str((root / "fiat").absolute()),
    #aggregation_areas=[
    #db.SpatialJoinModel(
    #    name="aggr_lvl_1",
    #    file=str(
    #        r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\aggregation_areas\aggr_lvl_1.geojson"
    #    ),
    #    field_name="name",
    #),
    #db.SpatialJoinModel(
    #    name="aggr_lvl_2",
    #    file=str(
    #        r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\aggregation_areas\aggr_lvl_2.geojson"
    #    ),
    #    field_name="name",
    #),
#],
    building_footprints=db.FootprintsOptions.OSM,
    fiat_buildings_name="buildings",
    fiat_roads_name="roads",
    bfe=db.SpatialJoinModel(
        file=str(r"C:\Users\rautenba\repos\Database\charleston_test\static\bfe\bfe.geojson"),
        name="bfe",
        field_name="bfe",
    ),
    svi=db.SviConfigModel(
        file=str(r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\svi\CDC_svi_2020.gpkg"),
        field_name="SVI",
        threshold=0.5,
    ),
    road_width=5,
    )

### **Step 3b**: Create a configuration file from a dictionary

In [ ]:
# Create the configuration file for the database builder for a simple FloodAdapt database. 

# all paths should be provided with forward slashes (/)
config = {"name": "charleston_example_advanced2",
 "database_path": r"C:\Users\rautenba\OneDrive - Stichting Deltares\Documents\Products\I1000642-021_FloodAdapt\Data_notebooks\data\nb1_database_builder\output\Database",
 "unit_system": "imperial",
 "gui": {"max_flood_depth": 5.0,
  "max_aggr_dmg": 1000000.0,
  "max_footprint_dmg": 250000.0,
  "max_benefits": 5000000.0},
 "infographics": True,
 "fiat": r"C:\Users\rautenba\OneDrive - Stichting Deltares\Documents\Products\I1000642-021_FloodAdapt\Data_notebooks\data\nb1_database_builder\fiat",
 #"aggregation_areas": [{"name": "aggr_lvl_1",
 #  "file": r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\aggregation_areas\aggr_lvl_1.geojson",
 #  "field_name": "name"},
 # {"name": "aggr_lvl_2",
 #  "file": r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\aggregation_areas\aggr_lvl_2.geojson",
 #  "field_name": "name"}],
 "building_footprints": "OSM",
 "fiat_buildings_name": "buildings",
 "fiat_roads_name": "roads",
 "bfe": {"name": "bfe",
  "file": r"C:\Users\rautenba\repos\Database\charleston_test\static\bfe\bfe.geojson",
  "field_name": "bfe"},
 "svi": {"name": None,
  "file": r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\fiat\svi\CDC_svi_2020.gpkg",
  "field_name": "SVI",
  "threshold": 0.5},
 "road_width": 5.0,
 "return_periods": [1, 2, 5, 10, 25, 50, 100],
 "sfincs_overland": {"name": r"C:\Users\rautenba\OneDrive - Stichting Deltares\Documents\Products\I1000642-021_FloodAdapt\Data_notebooks\data\nb1_database_builder\sfincs",
  "reference": "MSL",
  "vertical_offset": None},
 "sfincs_offshore": {"name": r"C:\Users\rautenba\repos\Database\charleston_test\static\templates\offshore",
  "reference": "MSL",
  "vertical_offset": {"value": 0.33, "units":"feet"}},
 "dem": {"filename": r"C:\Users\rautenba\repos\Database\charleston_test\static\dem\charleston_14m.tif",
  "units": "meters"},
 "excluded_datums": ["NAVD88"],
 "slr_scenarios": {"file": r"C:\Users\rautenba\repos\Database\charleston_test\static\slr\slr.csv",
  "relative_to_year": 2020},
 "scs": {"file": r"C:\Users\rautenba\repos\Database\charleston_test\static\scs\scs_rainfall.csv",
  "type": "type_3"},
 "tide_gauge": {"source":"noaa_coops",
  "description": "",
  "max_distance": {"value": 100.0, "units": "miles"}},
 "cyclones": True,
 "cyclone_basin": "NA",
 "obs_point": [{"name": "ashley_river",
   "description": "Ashley River - James Island Expy",
   "lat": 32.7765,
   "lon": -79.9543},
  {"name": 8665530,
   "description": "Charleston Cooper River Entrance",
   "ID": 8665530,
   "lat": 32.78,
   "lon": -79.9233}],
 "probabilistic_set": r"C:\Users\rautenba\repos\Database\charleston_test\input\events\test_set"}

# Save the configuration file
with open(root /"output"/"db_config_advanced.toml", "w") as f:
    toml.dump(config, f)

## **Step 3c**: Run the Database Builder

We are ready to run the Database Builder with the configuration that we just created above. First we are going to run **Option 3a** - in which we generated the configuration using the FloodAdapt classes. After we are going to run **Option 3b** - in which we manually created a configuration file from a dictionary. 

In [ ]:
# Run Option 2a -  DB-builder config from FloodAdapt classes
db_build = db.DatabaseBuilder(config_model)
db_build.build(overwrite= True)

# Run Option 2b -  DB-builder config from dictionary
config_path = root / "output" / "db_config_advanced.toml"
config = db.ConfigModel.read(config_path)
dbs = db.DatabaseBuilder(config)
dbs.build(overwrite = True)

Now you created a complete and advanced FloodAdapt Database. You can open the databases in the GUI and explore them further. 

<div>
<img src="../_static/images/example_chaleston2.png" width="900"/>
</div>

### **Finished!**

##### **Next steps**: Work with the database and build a scenario.

To learn how to set up a whole scnearios work through the following notebooks:
1. [Simple scenario](example1.ipynb)
2. Events
3. Projections
4. Measures
...
..
